In [11]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import re

import datetime
import time

import os
import warnings
warnings.filterwarnings(action='ignore')

In [12]:
os.listdir('C:/Users/KIMDAEUN/Desktop/gdp_data/기후금융/나우캐스트/')

['.ipynb_checkpoints',
 'nowcast_data_preprocessing',
 'nowcast_data_preprocessing.ipynb',
 'nowcast_data_preprocessing.zip',
 'nowcast_data_preprocessing_v2.ipynb',
 '가맹점 카드매출액(주,월).csv',
 '가맹점 현금매출액(월).csv',
 '신용카드이용금액(주).csv',
 '실업급여지급건수(월).csv',
 '실업급여지급금액(월).csv',
 '온라인채용 모집인원수(주).csv',
 '전자지급서비스 충전액(주).csv',
 '통신 모바일 인구이동량 통계(2022년 6월 2주차).csv',
 '통신 모바일 인구이동량 통계(2022년 6월 3주차).csv']

In [62]:
card = pd.read_csv('C:/Users/KIMDAEUN/Desktop/gdp_data/기후금융/나우캐스트/가맹점 카드매출액(주,월).csv')
cash = pd.read_csv('C:/Users/KIMDAEUN/Desktop/gdp_data/기후금융/나우캐스트/가맹점 현금매출액(월).csv')
fam_card = pd.read_csv('C:/Users/KIMDAEUN/Desktop/gdp_data/기후금융/나우캐스트/신용카드이용금액(주).csv')
un_num = pd.read_csv('C:/Users/KIMDAEUN/Desktop/gdp_data/기후금융/나우캐스트/실업급여지급건수(월).csv')
un_amount = pd.read_csv('C:/Users/KIMDAEUN/Desktop/gdp_data/기후금융/나우캐스트/실업급여지급금액(월).csv')
job_offer = pd.read_csv('C:/Users/KIMDAEUN/Desktop/gdp_data/기후금융/나우캐스트/온라인채용 모집인원수(주).csv')
fam_charge = pd.read_csv('C:/Users/KIMDAEUN/Desktop/gdp_data/기후금융/나우캐스트/전자지급서비스 충전액(주).csv')
mobile = pd.read_csv('C:/Users/KIMDAEUN/Desktop/gdp_data/기후금융/나우캐스트/통신 모바일 인구이동량 통계(2022년 6월 3주차).csv')

### preprocessing function

##### *두 함수 모두 실행

In [187]:
## all data preprocessing (monthly variation)
def input_df_date(df):
    date_list = []
    month_value_list = []

    try:
        df['year_month'] = df['Date'].str.slice(stop=7)
        
        for date in df.year_month.unique():
            four_weeks = df[df['year_month']==date].reset_index(drop=True)

            week_value_list = []
            week_value_list = four_weeks['전체'].to_list()

            month_value = np.log(sum([a for a in np.exp(week_value_list)]))
            month_value_list.append(month_value)
            date_list.append(date)
        
    except: ## mobile
        df = df.T
        df.columns = df.iloc[0]
        df = df.drop(df.index[0]).reset_index(drop=True)
        df = df.drop(['관내', '관외'], axis=1)

        df['year_month'] = df['전국 총계'].str.slice(stop=7)
        df['year_month'] = [df['year_month'][i].replace('.', '-') for i in range(len(df['year_month']))]
        df['전체'] = [int(df['합계'][i].replace(',', '')) for i in range(len(df['year_month']))]

        df.drop(['전국 총계', '합계'], axis=1, inplace=True)
        
        for date in df.year_month.unique():
            four_weeks = df[df['year_month']==date].reset_index(drop=True)

            week_value_list = []
            week_value_list = four_weeks['전체'].to_list()

            month_value = sum(week_value_list)
            month_value_list.append(month_value)
            date_list.append(date)
            
    df_month = pd.DataFrame(columns=['date', 'month_value'])
    df_month['date'] = date_list
    df_month['month_value'] = month_value_list
    
    df_month = df_month.set_index('date')
    
    return df_month

In [278]:
def input_date_alldata(start_year_month, end_year_month, *all_data):
    
    alldata_list = [input_df_date(sep) for sep in all_data]
    
    for i in range(len(alldata_list)):
        if alldata_list[i].loc['2020-02','month_value'] == 1.334110997696652:
            alldata_list[i].rename(columns={'month_value':'card'},inplace=True)
        elif alldata_list[i].loc['2020-02','month_value'] == 0.021544330000000007:
            alldata_list[i].rename(columns={'month_value':'cash'},inplace=True)
        elif alldata_list[i].loc['2020-02','month_value'] == 1.334864568442162:
            alldata_list[i].rename(columns={'month_value':'fam_card'},inplace=True)
        elif alldata_list[i].loc['2020-02','month_value'] == 0.005372560000000047:
            alldata_list[i].rename(columns={'month_value':'un_num'},inplace=True)
        elif alldata_list[i].loc['2020-02','month_value'] == 0.06512040000000005:
            alldata_list[i].rename(columns={'month_value':'un_amount'},inplace=True)
        elif alldata_list[i].loc['2020-02','month_value'] == 1.7124564799025555:
            alldata_list[i].rename(columns={'month_value':'job_offer'},inplace=True)
        elif alldata_list[i].loc['2020-02','month_value'] == 1.4243140303594255:
            alldata_list[i].rename(columns={'month_value':'fam_charg'},inplace=True)
        elif alldata_list[i].loc['2020-02','month_value'] == 1.4243140303594255:
            alldata_list[i].rename(columns={'month_value':'fam_charg'},inplace=True)
        elif alldata_list[i].loc['2020-02','month_value'] == 407998403:
            alldata_list[i].rename(columns={'month_value':'mobile'},inplace=True)
    
    term = pd.concat(alldata_list, axis=1).sort_values('date').reset_index()
    start = term.loc[term['date']==start_year_month].index.values
    end = term.loc[term['date']==end_year_month].index.values
    result = term.iloc[int(start):int(end)+1]
    
    result = result.set_index('date')
    
    return result

### function result

In [280]:
input_date_alldata('2019-01', '2020-10', card, cash, fam_card, un_num, un_amount, mobile)
#최종적으로 사용할 함수입니다. 사용하실 데이터를 *all_data에 개수 상관없이 넣으시면 됩니다.

,card,cash,fam_card,un_num,un_amount,mobile
date,,,,,,
2019-01,NaN,NaN,NaN,NaN,NaN,548705610
2019-02,NaN,NaN,NaN,NaN,NaN,429729149
2019-03,NaN,NaN,NaN,NaN,NaN,453005338
2019-04,NaN,NaN,NaN,NaN,NaN,460709859
2019-05,NaN,NaN,NaN,NaN,NaN,583466070
2019-06,NaN,NaN,NaN,NaN,NaN,460463571
2019-07,NaN,NaN,NaN,NaN,NaN,456753971
2019-08,NaN,NaN,NaN,NaN,NaN,562112879
2019-09,NaN,NaN,NaN,NaN,NaN,449666567


### mobile 주별->월별 합계

In [286]:
mobile = pd.read_csv('C:/Users/KIMDAEUN/Desktop/gdp_data/기후금융/나우캐스트/통신 모바일 인구이동량 통계(2022년 6월 3주차).csv')
mobile

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 172,Unnamed: 173,Unnamed: 174,Unnamed: 175,Unnamed: 176,Unnamed: 177,Unnamed: 178,Unnamed: 179,Unnamed: 180,Unnamed: 181
0,전국 총계,2019.01.1주차,2019.01.2주차,2019.01.3주차,2019.01.4주차,2019.01.5주차,2019.02.1주차,2019.02.2주차,2019.02.3주차,2019.02.4주차,...,2022.04.2주차,2022.04.3주차,2022.04.4주차,2022.05.1주차,2022.05.2주차,2022.05.3주차,2022.05.4주차,2022.06.1주차,2022.06.2주차,2022.06.3주차
1,합계,"107,286,640","109,856,167","110,428,194","110,926,323","110,208,286","101,977,829","110,300,567","108,797,073","108,653,680",...,"110,261,543","111,961,232","112,862,176","116,758,112","115,073,267","114,863,737","114,184,894","114,538,278","113,024,843","113,813,636"
2,관내,"73,267,421","75,223,704","75,492,823","75,842,477","74,530,848","66,169,221","75,359,200","73,779,392","72,528,942",...,"75,865,320","76,347,851","76,410,038","76,294,533","77,823,228","77,441,082","77,050,155","76,428,715","76,853,876","77,245,099"
3,관외,"34,019,219","34,632,463","34,935,371","35,083,846","35,677,438","35,808,608","34,941,367","35,017,681","36,124,738",...,"34,396,223","35,613,381","36,452,138","40,463,579","37,250,039","37,422,655","37,134,739","38,109,563","36,170,967","36,568,537"


In [295]:
mobile = pd.read_csv('C:/Users/KIMDAEUN/Desktop/gdp_data/기후금융/나우캐스트/통신 모바일 인구이동량 통계(2022년 6월 3주차).csv')
mobile = mobile.T
mobile.columns = mobile.iloc[0]
mobile = mobile.drop(mobile.index[0]).reset_index(drop=True)
mobile = mobile.drop(['관내', '관외'], axis=1)

mobile['year_month'] = mobile['전국 총계'].str.slice(stop=7)
mobile['year_month'] = [mobile['year_month'][i].replace('.', '-') for i in range(len(mobile['year_month']))]
mobile['전체'] = [int(mobile['합계'][i].replace(',', '')) for i in range(len(mobile['year_month']))]

mobile.drop(['전국 총계', '합계'], axis=1, inplace=True)

In [297]:
jan_list = mobile.iloc[0:5,1]
jan_list

0    107286640
1    109856167
2    110428194
3    110926323
4    110208286
Name: 전체, dtype: int64

In [298]:
sum(jan_list)

548705610

### test 주별->월별 수식

In [277]:
## 1월달 변동량으로 테스트 (수동계산)
test = card.iloc[0:4,1].to_list()
test

[-0.06402267, -0.00754951, 0.06603555, 0.00553663]

In [94]:
## exp
exp_test = np.exp(test)
exp_test

array([inf, inf, inf, inf])

In [9]:
np.exp(-0.06402267)

0.9379837351984223

In [10]:
## sum
sum_exp_test = np.sum(exp_test)
sum_exp_test

4.004279329932788

In [11]:
round(0.93798374+0.99247892+1.06826469+1.00555199, 10)

4.00427934

In [12]:
## log
log_sum_exp_test = np.log(sum_exp_test)
log_sum_exp_test

1.3873636217401453